In [ ]:
import numpy as np
from skimage import io, transform, color, filters
from matplotlib import pyplot as plt
from skimage.morphology import disk, ball
from skimage.filters import threshold_otsu, median

import cv2
import glob
from tqdm.notebook import tqdm

from numba import jit
import pandas as pd
from scipy.signal import argrelextrema
from statsmodels.tsa.api import SimpleExpSmoothing
from scipy.signal import find_peaks

from PIL import Image
from skimage import measure

In [ ]:
df=pd.read_csv('grades.csv')

df

## Header crop

In [ ]:
images_path='data/msk22/blanks_aligned_jpg/'
new_folder='data/msk22/headers_jpg/'

files_paths=glob.glob('data/msk22/blanks_aligned_jpg/*.jpg')
files_paths=[path.split('\\')[-1][:-4] for path in files_paths]

for id in tqdm(files_paths):

    img_path=images_path+id+'.jpg'
    img=io.imread(img_path)
    # msk22
    img =img[230:748,75:1390]

    # nov23
    # img = img[185:725, 90:1400]

    # msk 23
    # img = img[212:740, 70:1350]

    new_img_path=new_folder+id+'.jpg'
    io.imsave(new_img_path, img)

## Update metadata

In [ ]:
df=pd.read_csv('grades.csv')

new_df=[]

files_paths=glob.glob('data/nov23/headers_aligned_jpg/*.jpg')
files_paths=[path.split('\\')[-1][:-4] for path in files_paths]

tmp_df=df[df.type=='nov23']
for i in tqdm(range(len(tmp_df))):
    id=tmp_df.iloc[i]['id']
    if id in files_paths:
        new_df.append(tmp_df.iloc[i])

new_df=pd.DataFrame(new_df,columns=df.columns)
new_df['aligned_doc']=True
new_df.to_csv('grades_nov23.csv',index=False)

In [ ]:
df1=pd.read_csv('grades_nov23.csv')
df2=pd.read_csv('grades_msk23.csv')
df3=pd.read_csv('grades_msk22.csv')

df=pd.concat([df1,df2,df3])

df.to_csv('grades.csv')

In [ ]:
df

## Crop chars

In [ ]:
def crop(gra_f):
    mean_pixel_column=[]
    for j in range(gra_f.shape[1]):
        pixels=[]
        for i in range(gra_f.shape[0]):
            pixels.append(gra_f[i,j])
        mean_pixel_column.append(np.mean(pixels))

    deriv=np.gradient(mean_pixel_column)

    fit1 = SimpleExpSmoothing(deriv).fit(smoothing_level=0.2,optimized=False).fittedvalues

    x_min=find_peaks(fit1,)[0][0]

    min_extremax=argrelextrema(fit1, np.less)[0]

    if len(min_extremax)>1:
        x_max=min_extremax[-2]
    else:
        x_max=min_extremax[-1]

    return x_min, x_max


In [ ]:
@jit(nopython=True)
def frame_image(img, frame_width):
    b = frame_width # border size in pixel
    ny, nx = img.shape[0], img.shape[1] # resolution / number of pixels in x and y
    framed_img = np.ones((b+ny+b, b+nx+b))
    framed_img[b:-b, b:-b] = img
    return framed_img

In [ ]:
def rgb2smyk(bgr):
    bgrdash = bgr.astype(np.float)/255.

    # Calculate K as (1 - whatever is biggest out of Rdash, Gdash, Bdash)
    K = 1 - np.max(bgrdash, axis=2)

    # Calculate C
    C = (1-bgrdash[...,2] - K)/(1-K)

    # Calculate M
    M = (1-bgrdash[...,1] - K)/(1-K)

    # Calculate Y
    Y = (1-bgrdash[...,0] - K)/(1-K)

    K*=255
    K=K.astype(np.uint8)

    C*=255
    C=C.astype(np.uint8)

    M*=255
    M=M.astype(np.uint8)

    Y*=255
    Y=Y.astype(np.uint8)

    return C,M,Y,K

In [ ]:
tmp_img=median(img,ball(1))
(C, M, Y, K) = rgb2smyk(tmp_img)

thresh, binaryImage = cv2.threshold(Y, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

label_image = measure.label(binaryImage)

labels=np.sort(np.unique(label_image))
labels_num=[len(np.where(label_image==label)[0]) for label in labels][1:]

chars_labels=np.where(labels_num>np.mean(labels_num)/2)[0]+1
chars_labels=np.append(chars_labels, 0)

tmp_img=np.zeros(binaryImage.shape)
for label in chars_labels:
    tmp_img+=np.where(label_image==label, binaryImage, 0)

plt.figure(figsize=(15,6))
plt.imshow(tmp_img,cmap='gray')
plt.show()

In [ ]:
img=io.imread('msk22_ref_header_62263_5.jpg')
chars_num=9

coords={'surname':[82,46], 'name':[215, 46], 'code':[348, 46]}
width=45
height=55
step=6
boundary_adj=3

line_type='surname'
x,y=coords[line_type]
img=img[x-boundary_adj:x+boundary_adj+height, y-boundary_adj:y+boundary_adj+width*chars_num+step*(chars_num-1)]


tmp_img=median(img,ball(1))
(C, M, Y, K) = rgb2smyk(tmp_img)

thresh, binaryImage = cv2.threshold(Y, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

label_image = measure.label(binaryImage)

labels=np.sort(np.unique(label_image))
labels_num=[len(np.where(label_image==label)[0]) for label in labels][1:]

chars_labels=np.where(labels_num>np.mean(labels_num)/2)[0]+1
chars_labels=np.append(chars_labels, 0)

tmp_img=np.zeros(binaryImage.shape)
for label in chars_labels:
    tmp_img+=np.where(label_image==label, binaryImage, 0)

# y_min, y_max=crop(gra_f)
#
# N=20
# y_min=y_min-N
# if y_min<0:
#     y_min=0

# gra_f=gra_f[:, y_min:y_max+N]

# img_t=transform.resize(gra_f, (28,28,1))


fig,(ax1,ax2, ax3)=plt.subplots(1,3, figsize=(25,6))

ax1.imshow(img)
ax2.imshow(Y,cmap='gray')
ax3.imshow(tmp_img,cmap='gray')

plt.show()


In [ ]:
# msk22
# coords=[[83,46], [215, 46], [348, 46]]
# width=45
# height=55

# msk23
# coords=[[87,49], [220, 49], [353, 49]]
# width=45
# height=55

# nov23
coords=[[96,39], [228, 39], [363, 39]]
width=45
height=55

step=7

type='nov23'

df=pd.read_csv('grades.csv')

files_paths=glob.glob(f'data/{type}/headers_aligned_jpg/*.jpg')
files_ids=[path.split('\\')[-1][:-4] for path in files_paths]

for path in files_paths:
    img=io.imread(path)
